In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import IPython
from utils import visualize_annotations, classname_to_color_mapper

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width=1000,height=1000)
    IPython.display.display(i)

model = YOLO('runs/segment/train_export_large/weights/best.pt')   # load a pretrained model (recommended for training)

img_name = 'cubemap_P74_60e7f7bbe4021bd5e092ba94d6d502b7'
img_name_posx = img_name+'_posx'
img_name_negx = img_name+'_negx'
img_name_posy = img_name+'_posy'
img_name_negy = img_name+'_negy'
img_name_posz = img_name+'_posz'
img_name_negz = img_name+'_negz'
face_path_list = [img_name_posx,img_name_negx,img_name_posy,img_name_negy,img_name_posz,img_name_negz]
#results = model('datasets/Corrosao/images_full/2174.png', save = True)  # predict on an image

# Load your RGB image
image = Image.open('SISTEMAS_UFF/Images/cubemap_P74_fb48bf8c7a98b06384f1f93fcf027adb_posx.png')
width,height = image.size

image = image.convert("RGBA")

# Create a new image with an alpha channel
image_with_alpha = Image.new("RGBA", image.size)


complete_image = np.zeros((height*3,width*4,4))

# Create a drawing context for the image with an alpha channel
draw = ImageDraw.Draw(image_with_alpha)

# Define the points that form the polygon (replace with your polygon points)
polygon_points = [(100, 100), (200, 100), (200, 200), (100, 200)]

# Specify the fill color with transparency (e.g., (R, G, B, A))
fill_color = (255, 0, 0, 128)  # Red polygon with 50% opacity

# Draw the polygon on the image with transparency

for face_name in face_path_list:
    
    image_original = Image.open(f"SISTEMAS_UFF/Images/{face_name}.png")
    #image_original = Image.open(f"datasets/darwin_export_holes/images/test/{face_name}.png")

    if face_name.endswith("PosX"):
        pass
    elif face_name.endswith("NegX"):
        pass
    elif face_name.lower().endswith("negy"):
        image_original = image_original.rotate(270)
        pass
    elif face_name.lower().endswith("posy"):
        image_original = image_original.rotate(90)
        pass
    elif face_name.endswith("PosZ"):
        pass
    elif face_name.endswith("NegZ"):
        pass
    
    results = model(image_original, save = False)  # predict on an image
    
    image = image_original.convert("RGBA")

    # Create a new image with an alpha channel
    image_with_alpha = Image.new("RGBA", image.size)

    # Create a drawing context for the image with an alpha channel
    draw = ImageDraw.Draw(image_with_alpha)
    for result in results:
        
        if len(result) > 0:

            masks = result.masks.xy  # Masks object for segmentation masks outputs
            classes = result.boxes.cls  # Masks object for segmentation masks outputs
            for i in range(len(masks)):
                mask = masks[i]
                classe_index = int(classes[i].item())
                classe_name = result.names[classe_index]

                fill_color = classname_to_color_mapper(classe_name)
                outline_color = classname_to_color_mapper(classe_name,255)

                polygon_point_list = []
                for point in mask:
                    x,y = point
                    polygon_point_list.append((x,y))
                    
                if len(polygon_point_list) > 0:
                    draw.polygon(polygon_point_list, fill_color, outline_color)
    
        annotated_face = Image.alpha_composite(image, image_with_alpha)
        img = np.array(annotated_face) 
        
        height_pos = 0
        width_pos = 0
        if face_name.endswith("posx"):
            height_pos = 1
            width_pos = 2
            pass
        elif face_name.endswith("negx"):
            height_pos = 1
            width_pos = 0
            pass
        elif face_name.endswith("negy"):
            height_pos = 0
            width_pos = 1
            pass
        elif face_name.endswith("posy"):
            height_pos = 2
            width_pos = 1
            pass
        elif face_name.endswith("posz"):
            height_pos = 1
            width_pos = 1
            pass
        elif face_name.endswith("negz"):
            height_pos = 1
            width_pos = 3
            pass
        complete_image[height*height_pos:height*(height_pos+1),width*width_pos:width*(width_pos+1),:] = img
complete_image[..., :3] = complete_image[..., 2::-1]

#visualize_annotations(img_name,f"SISTEMAS_UFF/corrosion-v3/{img_name[8:]}.json")
imshow(complete_image)


In [ ]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageOps
import IPython
from utils import classname_to_color_mapper, draw_prediction_on_face, visualize_annotations



def imshow(img,size=(500,500)):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width =size[0],height =size[1])
    IPython.display.display(i)


FACES_LIST = ['_posx','_negx','_posy','_negy','_posz','_negz']

image_name = "cubemap_P74_5d1a4ffbcb8b43d55bb8af35519d6c41"


model = YOLO('runs/segment/train_corrosion_large_2//weights/best.pt')  # load a pretrained model (recommended for training)


def create_full_image_from_faces(face_path_list):
    width,height = 1344,1344

    # if face_path_list is the name of a face or image
    if type(face_path_list) == str:
        if face_path_list[-5:] in FACES_LIST:
            img_name = face_path_list[:-5]
        else:
            img_name = face_path_list
        if os.path.isfile(f"SISTEMAS_UFF/Images/{img_name}_posx.png"):
            face_path_list = [img_name+FACES_LIST[i] for i in range(len(FACES_LIST))]
        else:
            raise SystemExit(f"image {img_name} not found in default folder 'SISTEMAS_UFF/Images/'")
            return 0

    complete_image = np.zeros((height*3,width*4,3))
    for face_name in face_path_list:
        image_original = Image.open(f"SISTEMAS_UFF/Images/{face_name}.png")
        
        if face_name.lower().endswith("posx"):
            pass
        elif face_name.lower().endswith("negx"):
            pass
        elif face_name.lower().endswith("negy"):
            image_original = image_original.rotate(270)
            pass
        elif face_name.lower().endswith("posy"):
            image_original = image_original.rotate(90)
            pass
        elif face_name.lower().endswith("posz"):
            pass
        elif face_name.lower().endswith("negz"):
            pass
        
        img = np.array(image_original) 
        height_pos = 0
        width_pos = 0
        if face_name.endswith("posx"):
            height_pos = 1
            width_pos = 2
            pass
        elif face_name.endswith("negx"):
            height_pos = 1
            width_pos = 0
            pass
        elif face_name.endswith("negy"):
            height_pos = 0
            width_pos = 1
            pass
        elif face_name.endswith("posy"):
            height_pos = 2
            width_pos = 1
            pass
        elif face_name.endswith("posz"):
            height_pos = 1
            width_pos = 1
            pass
        elif face_name.endswith("negz"):
            height_pos = 1
            width_pos = 3
            pass
        complete_image[height*height_pos:height*(height_pos+1),width*width_pos:width*(width_pos+1),:] = img
    complete_image[..., :3] = complete_image[..., 2::-1]
    return complete_image


def extract_face_from_cubemap(cubemap):
    width,height = 1344,1344

    topo     = np.s_[height*0:height*1,width*1:width*2]
    esquerda = np.s_[height*1:height*2,width*0:width*1]
    centro   = np.s_[height*1:height*2,width*1:width*2]
    direita  = np.s_[height*1:height*2,width*2:width*3]
    contrario  = np.s_[height*1:height*2,width*3:width*4]
    base     = np.s_[height*2:height*3,width*1:width*2]

    lista_faces = [esquerda,centro,direita,contrario]

    face_1 = cubemap[esquerda]
    face_2 = cubemap[centro]
    face_3 = cubemap[direita]
    face_4 = cubemap[contrario]

    faces = [face_1,face_2,face_3,face_4]

    face_topo = cubemap[topo]
    face_base = cubemap[base]
    
    canvas = np.zeros((height*3,width*3,3))
    face_list = []

    for face_number in range(len(faces)):
        canvas[topo]     = np.rot90(face_topo, k=face_number, axes=(1,0))
        canvas[esquerda] = np.rot90(faces[face_number], k=0, axes=(1,0))
        canvas[centro]   = np.rot90(faces[(face_number+1)%4], k=0, axes=(1,0))
        canvas[direita]  = np.rot90(faces[(face_number+2)%4], k=0, axes=(1,0))
        canvas[base]     = np.rot90(face_base, k=-face_number, axes=(1,0))
        face_list.append(np.copy(canvas))
    
    canvas[topo]     = np.rot90(faces[3], k=2, axes=(1,0))
    canvas[esquerda] = np.rot90(faces[0], k=1, axes=(1,0))
    canvas[centro]   = np.rot90(face_topo, k=0, axes=(1,0))
    canvas[direita]  = np.rot90(faces[2], k=-1, axes=(1,0))
    canvas[base]     = np.rot90(faces[1], k=-0, axes=(1,0))
    face_list.append(np.copy(canvas))

    canvas[topo]     = np.rot90(faces[1], k=0, axes=(1,0))
    canvas[esquerda] = np.rot90(faces[0], k=-1, axes=(1,0))
    canvas[centro]   = np.rot90(face_base, k=0, axes=(1,0))
    canvas[direita]  = np.rot90(faces[2], k=1, axes=(1,0))
    canvas[base]     = np.rot90(faces[3], k=2, axes=(1,0))
    face_list.append(np.copy(canvas))

    return face_list


complete_image = create_full_image_from_faces(image_name)
imshow(complete_image)

face_list = extract_face_from_cubemap(complete_image)
        
width,height = 1344,1344

left_padding = 0
right_padding = 0

bot_padding = 0
top_padding = 0


padding_index = np.s_[height*1-left_padding:height*2+right_padding,width*1-bot_padding:width*2+top_padding]
    
central_face = np.s_[left_padding:height+right_padding,bot_padding:width+top_padding]


topo     = np.s_[height*0:height*1,width*1:width*2]
esquerda = np.s_[height*1:height*2,width*0:width*1]
centro   = np.s_[height*1:height*2,width*1:width*2]
direita  = np.s_[height*1:height*2,width*2:width*3]
contrario  = np.s_[height*1:height*2,width*3:width*4]
base     = np.s_[height*2:height*3,width*1:width*2]

face_position_list = [centro,direita,contrario,esquerda,topo,base]
face_name_list = ["_posz","_posx","_negz","_negx","_negy","_posy"]

canvas = np.zeros((3*height,4*width,4))
canvas_rotated_list = [np.zeros((height,width,4)) for _ in range(3)]

for face_index in range(len(face_list)):
    face = face_list[face_index]
    face_cutted = face[padding_index]
    face_position = face_position_list[face_index]
    #imshow(face_cutted)


    result_of_the_segmentation = model(face_cutted,save=False, verbose=False)
    mask = draw_prediction_on_face(face_cutted[..., 2::-1],result_of_the_segmentation[0])
    #imshow(mask[central_face])
    canvas[face_position] = mask[central_face]

    for i_1 in range(0):
        canvas_rotated = canvas_rotated_list[i_1]
        result_of_the_segmentation_rotated = model(np.rot90(face_cutted,i_1+1),save=False, verbose=False)
        mask_rotated = draw_prediction_on_face(np.rot90(face_cutted[..., 2::-1],i_1+1),result_of_the_segmentation_rotated[0])
        canvas_rotated_list[i_1] = np.rot90(mask_rotated[central_face],-(i_1+1))

    im = Image.fromarray(canvas[face_position][...,[2,1,0]].astype(np.uint8))
    #im.save(os.path.join(directory_padding,f"prediction_with_4_faces_{horizontal_padding}.png"))
    im.save(os.path.join("Patrick_images/new_results/",f"{image_name}_{face_name_list[face_index]}.png"))


imshow(canvas,(1000,1000))

for i_1 in range(0):
    imshow(canvas_rotated_list[i_1],(1000,1000))
